# 先找出表格頁碼
* camelot及pdfplumer皆無法正確識別有表格的pages，根據結果手動修正後，該pdf總共有59頁包含表格內容
table_page_num = [
    3,29,
    [34,35],
    [36,37],
    [38,39],40,41, 
    [46,47,48,49,50],
    51,54,62,64,65, # 64,65為無線條表格
    [69,70],
    72,
    [92,93,94],
    97,111,114,116,117,
    [122,123],
    [125,126,127,128,129],
    132,
    [133,134],
    [143,144,145,146,147,148,149],
    [157,158,159,160,161,162,163,164],
    166,171,172
]

In [4]:
import pdfplumber
from pprint import pprint
from tqdm import tqdm
import pandas as pd
import camelot
import psycopg2
import os
from openai import OpenAI
from dotenv import load_dotenv

client = os.getenv("OPENAI_API_KEY")

endpoint = "localhost"
database = "postgres"
username = "postgres"
password = "root"
port = 8080

connection = psycopg2.connect(
    dbname=database,
    host=endpoint,
    port=port,
    user=username,
    password="mysecretpassword" 
)
cur = connection.cursor()

In [5]:
# 建立流水編號 YYYYMMDDHHmmsssss000 Timestamp(17碼) + 隨機亂碼(3碼)
import datetime
import random

def generate_code():
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")[:-3]  # 17位timestamp
    random_code = ''.join(random.choices('0123456789', k=3))  # 3位random digit
    code = timestamp + random_code
    return code

# camelot/pdfplumber 讀取表格並抓取表格標題
* 部分表格需額外處理

In [ ]:
import camelot
import pdfplumber
import pandas as pd
import os
from operator import itemgetter

# 總頁碼(總共31張表格)
# TABLE_PAGE_SPEC = [
#     3, 29, [34, 35], [36, 37], [38, 39], 41,
#     [46, 47, 48, 49, 50], 51, 54, 62, 64, 65, [69, 70], 72,
#     [92, 93, 94], 97, 111, 114, 116, 117, [122, 123],
#     [125, 126, 127, 128, 129], 132, [133, 134],
#     [143, 144, 145, 146, 147, 148, 149],
#     [157, 158, 159, 160, 161, 162, 163, 164], 166, 171, 172
# ]

PDF_PATH = '../test.pdf'  

# # 單頁表格(64,65,171額外處理)
# TABLE_PAGE_SPEC = [
#     3, 29, 41, 51, 54, 62, 72, 97, 111, 114, 116, 117, 132, 166, 172
# ]

# # 無框線表格
# TABLE_PAGE_SPEC = [
#     64, 65
# ]

# # 單頁包含多個表格
# TABLE_PAGE_SPEC = [
#     171
# ]

# # 跨頁表格 (38,39,40額外處理)
# TABLE_PAGE_SPEC = [
#     [34, 35], [36, 37],
#     [46, 47, 48, 49, 50], [69, 70], 
#     [92, 93, 94], [122, 123],
#     [125, 126, 127, 128, 129], [133, 134],
#     [143, 144, 145, 146, 147, 148, 149],
#     [157, 158, 159, 160, 161, 162, 163, 164]
# ]

TABLE_PAGE_SPEC = [
    [38, 39, 40]
]

# Title finding parameters (pdfplumber)
VERTICAL_TOLERANCE = 10
HORIZONTAL_OVERLAP_RATIO = 0.05
LINE_GROUPING_Y_TOLERANCE = 3

# Camelot settings
CAMELOT_FLAVOR = 'lattice'  # 或 stream/hybrid/network, 根據表格類型選擇
CAMELOT_PROCESS_BACKGROUND = False # 是否處理背景線條 (對掃描件有用)

# pdfplumber 座標轉換和文本處理 
def convert_camelot_bbox_to_pdfplumber_bbox(camelot_bbox, page_height):
    x1_c, y1_c, x2_c, y2_c = camelot_bbox
    x0_p = x1_c
    top_p = page_height - y1_c
    x1_p = x2_c
    bottom_p = page_height - y2_c
    if top_p > bottom_p: top_p, bottom_p = bottom_p, top_p
    return (x0_p, top_p, x1_p, bottom_p)

def calculate_horizontal_overlap(bbox1, bbox2):
    overlap_start = max(bbox1[0], bbox2[0])
    overlap_end = min(bbox1[2], bbox2[2])
    return max(0, overlap_end - overlap_start)

def reconstruct_lines_from_words(words, y_tolerance=3):
    lines = []
    if not words: return lines
    sorted_words = sorted(words, key=itemgetter('top', 'x0'))
    current_line_words = []
    current_line_top = -1
    for word in sorted_words:
        word_top = word['top']
        if not current_line_words or abs(word_top - current_line_top) <= y_tolerance:
            if not current_line_words: current_line_top = word_top
            current_line_words.append(word)
            current_line_top = min(current_line_top, word_top)
        else:
            if current_line_words:
                current_line_words.sort(key=itemgetter('x0'))
                line_text = " ".join(w['text'] for w in current_line_words)
                line_x0 = min(w['x0'] for w in current_line_words)
                line_x1 = max(w['x1'] for w in current_line_words)
                line_bottom = max(w['bottom'] for w in current_line_words)
                lines.append({'text': line_text, 'bbox': (line_x0, current_line_top, line_x1, line_bottom)})
            current_line_words = [word]
            current_line_top = word['top']
    if current_line_words:
        current_line_words.sort(key=itemgetter('x0'))
        line_text = " ".join(w['text'] for w in current_line_words)
        line_x0 = min(w['x0'] for w in current_line_words)
        line_x1 = max(w['x1'] for w in current_line_words)
        line_bottom = max(w['bottom'] for w in current_line_words)
        lines.append({'text': line_text, 'bbox': (line_x0, current_line_top, line_x1, line_bottom)})
    return lines

# 標題查找函數 - 接收 pdfplumber 頁面和 *Camelot* bbox
def get_line_above_table_pdfplumber(plumber_page, table_bbox_camelot,
                                    tolerance=5, min_overlap_ratio=0.3,
                                    line_y_tolerance=3):
    """
    使用 pdfplumber 頁面物件和 Camelot bbox 尋找標題行。
    在函數內部進行座標轉換。
    """
    if not plumber_page or not table_bbox_camelot:
        return None

    page_height = plumber_page.height
    try:
        table_bbox_plumber = convert_camelot_bbox_to_pdfplumber_bbox(table_bbox_camelot, page_height)
    except Exception as e:
        print(f"      錯誤：轉換 Camelot Bbox 時出錯: {e}")
        return None

    table_top_y = table_bbox_plumber[1]
    table_width = table_bbox_plumber[2] - table_bbox_plumber[0]

    # 後續邏輯與純 pdfplumber 版本中的 get_line_above... 函數相同
    words = plumber_page.extract_words(keep_blank_chars=True, use_text_flow=True, y_tolerance=line_y_tolerance)
    lines = reconstruct_lines_from_words(words, y_tolerance=line_y_tolerance)
    candidate_lines = []
    for line in lines:
        line_bbox = line['bbox']
        line_bottom_y = line_bbox[3]
        line_width = line_bbox[2] - line_bbox[0]
        if table_top_y - tolerance <= line_bottom_y < table_top_y:
            overlap_width = calculate_horizontal_overlap(line_bbox, table_bbox_plumber)
            if table_width > 1 and line_width > 1:
                overlap_ratio = overlap_width / table_width
                if overlap_ratio >= min_overlap_ratio:
                    line_text = line['text'].strip()
                    if line_text:
                        candidate_lines.append((line_bottom_y, line_text, line_bbox))
    if candidate_lines:
        candidate_lines.sort(key=itemgetter(0), reverse=True)
        return candidate_lines[0][1]
    return None

# --- Main Execution ---
if __name__ == "__main__":
    if not os.path.exists(PDF_PATH):
        print(f"錯誤：PDF 文件 '{PDF_PATH}' 不存在。請檢查路徑。")
    else:
        print(f"正在處理 PDF: {PDF_PATH} 使用 Camelot ({CAMELOT_FLAVOR}) + pdfplumber")
        results = [] 

        # 使用 pdfplumber 打開一次，用於後續所有頁面的標題查找
        try:
            with pdfplumber.open(PDF_PATH) as pdf:
                # 遍歷 TABLE_PAGE_SPEC 中的每個項目
                for item_index, page_spec_item in enumerate(TABLE_PAGE_SPEC):

                    title = None
                    table_data_df = None # 現在儲存 DataFrame
                    processed_pages_list = []
                    camelot_bbox_ref = None # 記錄用於查找標題的 Camelot bbox
                    item_label = f"項目 {item_index + 1}"

                    try:
                        if isinstance(page_spec_item, int):
                            # --- 處理單頁表格 ---
                            page_num = page_spec_item
                            item_label = f"項目 {item_index + 1} (單頁: {page_num})"
                            print(f"\n--- {item_label} ---")
                            processed_pages_list = [page_num]

                            # 1. 使用 Camelot 處理單頁
                            tables_camelot = camelot.read_pdf(
                                PDF_PATH,
                                pages=str(page_num),
                                flavor=CAMELOT_FLAVOR,
                                process_background=CAMELOT_PROCESS_BACKGROUND,
                                suppress_stdout=True, # 減少輸出
                                parallel=True
                            )

                            if tables_camelot.n == 0:
                                print(f"  Camelot 未在頁碼 {page_num} 偵測到表格。")
                                continue # 跳到下一個項目

                            # 處理該頁找到的表格
                            table_cam = tables_camelot[0]
                            camelot_bbox_ref = table_cam._bbox # 用於查找標題
                            print(f"  Camelot 偵測到表格 (頁碼 {table_cam.page}, bbox: {camelot_bbox_ref})。")

                            # 2. 使用 pdfplumber 查找標題
                            if page_num >= 1 and page_num <= len(pdf.pages):
                                plumber_page = pdf.pages[page_num - 1]
                                title = get_line_above_table_pdfplumber(
                                    plumber_page, camelot_bbox_ref, VERTICAL_TOLERANCE,
                                    HORIZONTAL_OVERLAP_RATIO, LINE_GROUPING_Y_TOLERANCE)
                                print(f"    標題 (pdfplumber): '{title}'" if title else "    未能找到標題。")
                            else:
                                print(f"    警告：無法訪問 pdfplumber 頁面 {page_num} 來查找標題。")

                            # 3. 從 Camelot 獲取數據
                            table_data_df = table_cam.df
                            print(f"    提取到 {len(table_data_df)} 行數據 (來自 Camelot)。")

                        elif isinstance(page_spec_item, list):
                            # --- 處理跨頁表格 ---
                            page_list = page_spec_item
                            item_label = f"項目 {item_index + 1} (跨頁: {page_list})"
                            print(f"\n--- {item_label} ---")
                            processed_pages_list = page_list
                            pages_str = ','.join(map(str, page_list))

                            # 1. 使用 Camelot 處理所有相關頁面
                            tables_camelot = camelot.read_pdf(
                                PDF_PATH,
                                pages=pages_str,
                                flavor=CAMELOT_FLAVOR,
                                process_background=CAMELOT_PROCESS_BACKGROUND,
                                suppress_stdout=True,
                                strip_text='\n', # 移除換行符號
                                parallel=True

                            )

                            if tables_camelot.n == 0:
                                print(f"  Camelot 未在頁碼 {pages_str} 偵測到任何表格。")
                                continue

                            # 過濾出確實屬於本次處理頁碼列表的表格
                            relevant_tables = [tbl for tbl in tables_camelot if tbl.page in page_list]

                            if not relevant_tables:
                                print(f"  Camelot 在頁碼 {pages_str} 找到 {tables_camelot.n} 個表格，但沒有屬於 {page_list} 的。")
                                continue

                            print(f"  Camelot 在頁碼 {pages_str} 找到 {len(relevant_tables)} 個相關表格片段。")

                            # 尋找第一頁的表格片段以查找標題
                            first_page_num = page_list[0]
                            first_page_table = None
                            for tbl in relevant_tables:
                                if tbl.page == first_page_num:
                                    first_page_table = tbl
                                    break

                            if not first_page_table:
                                print(f"  警告：未能在第一頁 ({first_page_num}) 找到對應的 Camelot 表格片段。無法查找標題。")
                            else:
                                camelot_bbox_ref = first_page_table._bbox
                                print(f"  將使用第一頁 (頁碼 {first_page_num}) 的表格片段 (bbox: {camelot_bbox_ref}) 來查找標題。")
                                # 2. 使用 pdfplumber 查找標題 (基於第一頁片段)
                                if first_page_num >= 1 and first_page_num <= len(pdf.pages):
                                    plumber_page = pdf.pages[first_page_num - 1]
                                    title = get_line_above_table_pdfplumber(
                                        plumber_page, camelot_bbox_ref, VERTICAL_TOLERANCE,
                                        HORIZONTAL_OVERLAP_RATIO, LINE_GROUPING_Y_TOLERANCE)
                                    print(f"    標題 (pdfplumber): '{title}'" if title else "    未能找到標題。")
                                else:
                                    print(f"    警告：無法訪問 pdfplumber 頁面 {first_page_num} 來查找標題。")

                            # 3. 合併所有相關表格片段的數據 (來自 Camelot)
                            if relevant_tables:
                                try:
                                    # 按頁碼排序表格片段以確保合併順序正確
                                    relevant_tables.sort(key=lambda t: t.page)
                                    table_data_df = pd.concat([tbl.df for tbl in relevant_tables], ignore_index=True)
                                    print(f"    合併了 {len(relevant_tables)} 個表格片段，共 {len(table_data_df)} 行數據 (來自 Camelot)。")
                                except Exception as e:
                                    print(f"    合併 Camelot DataFrame 時出錯: {e}")
                                    table_data_df = None # 合併失敗
                            else:
                                table_data_df = None # 沒有相關表格可以合併


                        else:
                            print(f"警告：跳過無法識別的項目 {page_spec_item}")
                            continue

                        results.append({
                            "item_label": item_label,
                            "pages_processed": processed_pages_list,
                            "title": title,
                            "table_data_df": table_data_df, # 儲存 DataFrame
                            "camelot_bbox_reference": camelot_bbox_ref # 用於查找標題的 bbox
                        })

                    except Exception as e:
                        print(f"處理項目 {item_label} 時發生未預期的錯誤: {e}")

        except Exception as e:
            print(f"打開或處理 PDF '{PDF_PATH}' 時發生嚴重錯誤: {e}")

        # --- 處理/輸出結果 ---
        print("\n" + "="*30 + " 最終結果 " + "="*30)
        if not results:
            print("沒有處理任何有效的表格項目。")
        else:
            for i, result in enumerate(results):
                print(f"\n結果 {i+1}: {result['item_label']}")
                print(f"  處理頁碼: {result['pages_processed']}")
                print(f"  標題 (pdfplumber): {result['title'] if result['title'] else '未找到'}")
                print(f"  參考 Bbox (Camelot): {result['camelot_bbox_reference']}")
                df = result['table_data_df']
                total_rows = len(df) if df is not None else 0
                print(f"  表格數據 (來自 Camelot, 共 {total_rows} 行):")
                if df is not None and not df.empty:
                    # 使用 to_string 打印 DataFrame 預覽
                    print(df.to_string(max_rows=10)) # 顯示前 5 行和後 5 行
                elif df is not None and df.empty:
                    print("    (空表格)")
                else:
                    print("    無數據")
                print("-" * 40)

        print("\n處理完成。")

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong pointing object 68 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong 

正在處理 PDF: ../test.pdf 使用 Camelot (lattice) + pdfplumber

--- 項目 1 (跨頁: [38, 39, 40]) ---


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

  Camelot 在頁碼 38,39,40 找到 2 個相關表格片段。
  將使用第一頁 (頁碼 38) 的表格片段 (bbox: (85.4278221208666, 97.64062877871827, 757.5720068415051, 487.4834340991536)) 來查找標題。


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

    標題 (pdfplumber): '1.4  相關人員學經歷'
    合併了 2 個表格片段，共 11 行數據 (來自 Camelot)。

============================== 最終結果 ==============================

結果 1: 項目 1 (跨頁: [38, 39, 40])
  處理頁碼: [38, 39, 40]
  標題 (pdfplumber): 1.4  相關人員學經歷
  參考 Bbox (Camelot): (85.4278221208666, 97.64062877871827, 757.5720068415051, 487.4834340991536)
  表格數據 (來自 Camelot, 共 11 行):
      0                          1         2                        3                                                                                                                                                                                                                                                                                                                                                                     4
0    姓名                      單位/職稱  年資  最高學歷                                                                                                                                                                               

In [ ]:
# 檢查標題及表格是否正確抓取
from IPython.display import display

count = 0
for i in results:
    if i['title'] == None:
        count = count + 1
    print(i['title'])
    # display(i['table_data_df'])
print('標題缺失: ',count)

1.4  相關人員學經歷
標題缺失:  0


In [44]:
# 分別將跨頁表格匯入向量資料庫(title + markdown)
# 如果title為None則透過LLM生成title

is_single = "N"
descripe = None

for i in tqdm(results):
    _id = generate_code()
    if i['title'] != None:
        table_content = i['title'] + i['table_data_df'].to_markdown()
    else:
        generate_title = client.chat.completions.create(
            # model="gpt-4o-mini",
            model="gpt-4.1-mini",
            messages=[
                {
                    "role": "user",
                    "content": "請根據以下的markdown表格內容，生成20字以內的標題。\n表格內容:%s" % i['table_data_df'].to_markdown() 
                }
            ]
        )
        table_content = generate_title.choices[0].message.content + i['table_data_df'].to_markdown()

    markdown_embedding = client.embeddings.create(
        model="text-embedding-3-large",
        input=table_content,
        encoding_format="float"
    ).data[0].embedding
    data_insert = [
        (_id,table_content,descripe,markdown_embedding,is_single)
    ]
    insert_args = ",".join(cur.mogrify("(%s,%s,%s,%s,%s)",i).decode("utf-8") for i in data_insert)
    cur.execute("INSERT INTO public.table_markdown_v2 VALUES " + (insert_args))
    connection.commit()

# cur.close()
# connection.close()
print("INSERT SUCCESSFULLY")

100%|██████████| 10/10 [00:16<00:00,  1.65s/it]

INSERT SUCCESSFULLY


In [46]:
# # 檢查標題及表格是否正確抓取(手動修正部分標題)
# # 有問題的index=[1,8,13,14,15]

# count = 0
# for i in results:
#     if i['title'] == None:
#         count = count + 1
#     print(i['title'])
#     # display(i['table_data_df'])
# print('標題缺失: ',count)

In [45]:
# # 根據回傳資訊手動修正缺失標題
# missing_title_idx = [1,8,13,14,15]
# missing_title = ['附件一覽表','附件一覽表','','區域安排','講師介紹']

# # 使用 zip 將索引和標題配對遍歷
# for idx, title in zip(missing_title_idx, missing_title):
#     results[idx]['title'] = title
#     print(title)

# # 檢查是否替換
# count = 0
# for i in results:
#     if i['title'] == None:
#         count = count + 1
#     # print(i['title'])
#     # display(i['table_data_df'])
# print('標題缺失: ',count)

In [50]:
# 分別將單頁表格匯入向量資料庫(title + markdown) 15個單頁表格
is_single = "Y"
descripe = None

for i in tqdm(results):
    _id = generate_code()
    if i['title'] != None:
        table_content = i['title'] + i['table_data_df'].to_markdown()
    else:
        generate_title = client.chat.completions.create(
            # model="gpt-4o-mini",
            model="gpt-4.1-mini",
            messages=[
                {
                    "role": "user",
                    "content": "請根據以下的markdown表格內容，生成20字以內的標題。\n表格內容:%s" % i['table_data_df'].to_markdown() 
                }
            ]
        )
        table_content = generate_title.choices[0].message.content + i['table_data_df'].to_markdown()

    markdown_embedding = client.embeddings.create(
        model="text-embedding-3-large",
        input=table_content,
        encoding_format="float"
    ).data[0].embedding
    data_insert = [
        (_id,table_content,descripe,markdown_embedding,is_single)
    ]
    insert_args = ",".join(cur.mogrify("(%s,%s,%s,%s,%s)",i).decode("utf-8") for i in data_insert)
    cur.execute("INSERT INTO public.table_markdown_v2 VALUES " + (insert_args))
    connection.commit()

# cur.close()
# connection.close()
print("INSERT SUCCESSFULLY")

100%|██████████| 15/15 [00:18<00:00,  1.26s/it]

INSERT SUCCESSFULLY


In [64]:
# 例外單頁表格(64,65)
tables = camelot.read_pdf('./test.pdf',pages='64,65',flavor="stream",parallel=True,strip_text='\n')
fix_table_p64 = tables[0].df.drop(index=[0,1])
fix_title_p64 = client.chat.completions.create(
            # model="gpt-4o-mini",
            model="gpt-4.1-mini",
            messages=[
                {
                    "role": "user",
                    "content": "請根據以下的markdown表格內容，生成20字以內的標題。\n表格內容:%s" % fix_table_p64.to_markdown() 
                }
            ]
        ).choices[0].message.content
print(fix_title_p64)
display(fix_table_p64) 

fix_table_p65 = tables[1].df.drop(index=[0,1])
fix_title_p65 = client.chat.completions.create(
            # model="gpt-4o-mini",
            model="gpt-4.1-mini",
            messages=[
                {
                    "role": "user",
                    "content": "請根據以下的markdown表格內容，生成20字以內的標題。\n表格內容:%s" % fix_table_p65.to_markdown() 
                }
            ]
        ).choices[0].message.content
print(fix_title_p65)
display(fix_table_p65)


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong pointing object 68 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong 

化學與環境資料及QSAR模型統計


,0,1,2,3
2,分類,化學物數,實驗測量資料數,QSAR 模型數
3,物理化學特性,"50,771","251,311",28
4,環境宿命與傳輸,"15,610","177,909",41
5,環境毒理資訊,"23,678","1,423,889",39
6,人類健康危害,"45,931","1,367,583",146


小鼠與大鼠毒性預測數據總覽


,0,1,2,3,4,5
2,編號,物種,預測值,適用範圍?,終點,暴露途徑
3,1,小鼠,500 mg/kg bdwt/d,是,LD50,腹腔注射
4,2,小鼠,無預測,否,LD50,靜脈注射
5,3,小鼠,2.2E3 mg/kg bdwt/d,是,LD50,口服
6,4,小鼠,1.7E3 mg/kg bdwt/d,是,LD50,皮下組織吸收
7,5,大鼠,690 mg/kg bdwt/d,是,LD50,腹腔注射
8,6,大鼠,2.2E3 mg/kg bdwt/d,是,LD50,口服


In [65]:
# 分別將例外單頁表格(p64,65)匯入向量資料庫(title + markdown)
title = [fix_title_p64,fix_title_p65]
table = [fix_table_p64,fix_table_p65]
is_single = "N"
descripe = None

for t1,t2 in tqdm(zip(title,table)):
    _id = generate_code()
    table_content = t1 + t2.to_markdown()
    markdown_embedding = client.embeddings.create(
        model="text-embedding-3-large",
        input=table_content,
        encoding_format="float"
    ).data[0].embedding
    data_insert = [
        (_id,table_content,descripe,markdown_embedding,is_single)
    ]
    insert_args = ",".join(cur.mogrify("(%s,%s,%s,%s,%s)",i).decode("utf-8") for i in data_insert)
    cur.execute("INSERT INTO public.table_markdown_v2 VALUES " + (insert_args))
    connection.commit()

# cur.close()
# connection.close()
print("INSERT SUCCESSFULLY")


2it [00:02,  1.09s/it]

INSERT SUCCESSFULLY


In [75]:
# 例外跨跨頁表格(38,39,40)
tables = camelot.read_pdf('./test.pdf',pages='38,39',flavor="lattice",parallel=True,strip_text='\n')
tb = pd.concat([tables[0].df,tables[1].df])

# 用stream擷取p40後合併
tables = camelot.read_pdf('./test.pdf',pages='40',flavor="stream",parallel=True,strip_text='\n')
fix_table_p40 = tables[0].df.drop(index=[0,1,3])
tb = pd.concat([tb,fix_table_p40])
tb_title = "1.4  相關人員學經歷"
display(tb)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong pointing object 68 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong 

,0,1,2,3,4
0,姓名,單位/職稱,年資 最高學歷,,曾參與計畫名稱
1,曾＊＊,臺灣大學/教授,16,美國伊利諾大學芝加哥分校 藥物化學及生藥學博士,l 110-111 年綠色化學-安全替代整合性政策研究計畫(2021-2022) l 全自...
2,蘇＊＊,英屬開曼群島商思捷優達股份有限公司臺灣分公司/顧問,3,臺灣大學電資學院博士,l 110-111 年綠色化學-安全替代整合性政策研究計畫(2021-2022) l 基於...
0,姓名,單位/職稱,年資 最高學歷,,曾參與計畫名稱
1,王＊＊,臺灣大學/博士候選人,-,臺灣大學生醫電資所,l 110-111 年綠色化學-安全替代整合性政策研究計畫(2021-2022) l 再生...
2,黎＊＊,臺灣大學/博士候選人,-,臺灣大學生醫電資所,l 再生醫學科技發展計畫--全身安全性異體移植策略之細胞治療產品開發:標靶醫療未滿足慢性發...
3,杜＊＊,臺灣大學/博士生,-,臺灣大學生醫電資所,
4,許＊＊,臺灣大學/碩士生,-,臺灣大學生醫電資所,
5,王＊＊,臺灣大學/碩士生,-,臺灣大學生醫電資所,
6,高＊＊,臺灣大學/碩士生,-,臺灣大學生醫電資所,


In [77]:
# 分別將例外跨頁表格匯入向量資料庫(title + markdown)
title = tb_title
table = tb
is_single = "N"
descripe = None
_id = generate_code()
table_content = title + table.to_markdown()

markdown_embedding = client.embeddings.create(
    model="text-embedding-3-large",
    input=table_content,
    encoding_format="float"
).data[0].embedding

data_insert = [
    (_id,table_content,descripe,markdown_embedding,is_single)
]
insert_args = ",".join(cur.mogrify("(%s,%s,%s,%s,%s)",i).decode("utf-8") for i in data_insert)
cur.execute("INSERT INTO public.table_markdown_v2 VALUES " + (insert_args))
connection.commit()

print("INSERT SUCCESSFULLY")

INSERT SUCCESSFULLY


In [15]:
# p171單頁表格
import camelot
# "29,111,166,171,172"
tables = camelot.read_pdf("./test.pdf",suppress_stdout=True,pages="171")

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong pointing object 68 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong 

In [21]:
p171_title = []
p171_table = []
# len(tables)
for t in tables:
    table = t.df.to_markdown()

    completion = client.chat.completions.create(
        # model="gpt-4o-mini",
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "user",
                "content": "請根據以下的markdown表格內容，生成20字以內的標題。\n表格內容:%s" % table 
            }
        ]
    )
    p171_title.append(completion.choices[0].message.content)
    p171_table.append(table)
    print(completion.choices[0].message.content)

北區臺北市辦理場次與容量簡表
北區國立臺灣大學教室資訊
毒化物專家研討會場次與時間安排


In [23]:
# 分別將例外p171表格匯入向量資料庫(title + markdown)
title = p171_title
table = p171_table
is_single = "Y"
descripe = None

for t1,t2 in tqdm(zip(title,table)):
    _id = generate_code()
    table_content = t1 + t2
    markdown_embedding = client.embeddings.create(
        model="text-embedding-3-large",
        input=table_content,
        encoding_format="float"
    ).data[0].embedding
    data_insert = [
        (_id,table_content,descripe,markdown_embedding,is_single)
    ]
    insert_args = ",".join(cur.mogrify("(%s,%s,%s,%s,%s)",i).decode("utf-8") for i in data_insert)
    cur.execute("INSERT INTO public.table_markdown_v2 VALUES " + (insert_args))
    connection.commit()

cur.close()
connection.close()
print("INSERT SUCCESSFULLY")

3it [00:04,  1.57s/it]

INSERT SUCCESSFULLY


# LLMsherpa(find sections)
根據每個段落再切向量table

In [ ]:
from llmsherpa.readers import LayoutPDFReader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
import itertools

# 讀取文件
llmsherpa_api_url = "http://127.0.0.1:5010/api/parseDocument?renderFormat=all" # "&applyOcr=yes""&useNewIndentParser=true"
pdf_url = "../test.pdf" 
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)

# 將文件中的段落依序切分後建立langchain documents，保證所有的chunk包含段落的所有資訊
result = []
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50,length_function=len,separators=["\n\n", "", "。"]) # overlap between 10%~15%
text_splitter = CharacterTextSplitter(chunk_size=300,chunk_overlap=45,length_function=len,separator="") # overlap between 10%~15%s
for section in doc.sections():
    chunk = section.to_text(include_children=True, recurse=True)
    section_num = section.page_idx
    content = text_splitter.create_documents(texts=[chunk],metadatas=[{"source": section_num}])
    result.append(content)

# 將巢狀list解開為單一list
flat_list = list(itertools.chain.from_iterable(result))
# flat_list
print(len(flat_list))

1921


In [46]:
# 匯入資料庫
# 整份文件切斷後押入向量資料庫
is_table = "N"
# table_content = None

for i in tqdm(flat_list):
    _id = generate_code()
    text_segment = i.page_content
    openai_embedding = client.embeddings.create(
        model="text-embedding-3-large",
        input=text_segment,
        encoding_format="float"
    ).data[0].embedding
    data_insert = [
        (_id,text_segment,is_table,openai_embedding)
    ]
    insert_args = ",".join(cur.mogrify("(%s,%s,%s,%s)",i).decode("utf-8") for i in data_insert)
    cur.execute("INSERT INTO public.section_content_charsplitt VALUES " + (insert_args))
    connection.commit()

cur.close()
connection.close()
print("INSERT SUCCESSFULLY")


100%|██████████| 1921/1921 [27:08<00:00,  1.18it/s]  

INSERT SUCCESSFULLY


In [49]:
# 向量搜尋
openai_embedding = client.embeddings.create(
        model="text-embedding-3-large",
        input="哪一些化學物質會對內分泌系統造成干擾?",
        encoding_format="float"
    ).data[0].embedding

cur.execute("SELECT (1 - (openai_embedding <=> " + "'" + str(openai_embedding) + "'" + ")) AS similarity,text_segment FROM public.section_content_charsplitt ORDER BY similarity DESC limit 10;")
rows = cur.fetchall()

print("=========== TOP 3 ===========")
for row in rows:
    print(row)

llm_reference = []
user_query = "哪一些化學物質會對內分泌系統造成干擾"

for row in rows:
    llm_reference.append(row[1])

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "請根據以下檢索到的內容來回答user提出的問題，若不知道請回答不知道即可。\n檢索內容:%s" % llm_reference + "問題:%s" % user_query
        }
    ]
)
print("=========== LLM Answer ===========")
print(completion.choices[0].message.content)

=========== TOP 3 ===========
(0.6348437884543079, '組織公佈的「疑似內分泌干擾物」清單中彙整出一份化學 品工作清單，並輔以科學文獻的搜索，找出提及特定化學品內 分泌干擾作用的報告和論文。為了盡可能確保清單的全面性， 清單草案會與主要利益相關者（包括來自政府、行業和非政府 組織的代表）開會討論。有關化學品對人類、其他脊椎動物和 無脊椎動物內分泌影響之資料都會被蒐集納入資料庫中，另外， 若還有每種化學品在環境中的持久性和在暴露於該環境下之生 物體中積累的可能性（即生物積累）也會一併蒐集。\n2. 對現有的資訊進行審查，找出可能在環境中具有高度持久性 （即抗分解性）或由工業界大量生產的化學品（即每年超過\n1000 噸）。採用這兩個標準是因為在這兩種條件')
(0.6211277472300691, '暴 露的化學品給予最低關注。\n歐洲執委會分析了由各組織公佈的 564 個疑似內分泌干擾物， 最終歸類有 194 個第一類化學品與 126 個第二類化學品，並彙整成 一個資料庫檔案。 （二） Perkins and Will 預防性清單(Perkins and Will Precautionary List)是由建 築設計公司 Perkins and Will 所制定的一份清單，旨在協助建築師、設 計師和業主選擇更環保和健康的建材和產品。這份清單列出了一系 列被認為可能對人體健康或環境造成潛在風險的化學品物質，並提 供了可替代的更環保和健康的選擇。Perkins and Will 預防性清單的目')
(0.6202870607376143, '擾活動 的證據 第 2 類：具有至少一些與體外內分泌干擾有關的生物活動證據 第 3a 類：沒有內分泌干擾活動的證據 第 3b 類：資料不足 4.\n對於在步驟三中歸於第一類的化學品審查現有的資訊，判斷人 類或野生動物實際接觸到的可能性。對人類或野生動物預計會 暴露的化學品給予最高關注；對人類預計不會暴露但野生動物 可能暴露的化學品給予中等關注；對人類或野生動物都不會暴 露的化學品給予最低關注。\n歐洲執委會分析了由各組織公佈的 564 個疑似內分泌干擾物， 最終歸類有 194 個第一類化學品與 126 個第二類化學品，並彙整成 一個資料庫檔案。 （二） Perkins and Wil